In [1]:
import pandas as pd
from datasets import load_dataset

/Users/rohanbhardwaj/Documents/ml_projects/Legal Document Summariser/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
#Load Dataset
dataset = load_dataset("json",
                       data_files = {"train": "us_train_data_final_OFFICIAL.jsonl",
                                     "test": "us_test_data_final_OFFICIAL.jsonl"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [24]:
dataset["train"][0]

{'bill_id': '107_hr2256',
 'text': "SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``Border Hospital Survival and Illegal \nImmigrant Care Act''.\n\nSEC. 2. FINDINGS.\n\n    The Congress finds as follows:\n            (1) Immigration is a Federal responsibility.\n            (2) The Immigration and Naturalization Service does not \n        take into custody all aliens who are unlawfully present in the \n        United States.\n            (3) Section 1867 of the Social Security Act (42 U.S.C. \n        1395dd) and State laws require that, if any individual (whether \n        or not lawfully present in the United States) comes to a \n        hospital and the hospital determines that the individual has an \n        emergency medical condition, the hospital must provide either, \n        within the staff and facilities available at the hospital, for \n        such further medical examination and such treatment as may be \n        required to stabilize the medical condition, o

In [27]:
# Checking first 5 bills
for i in range(5):
    print(dataset["train"][i])
    #print("Bill text:", dataset["train"][i]["text"][:300], "...\n")
    #print("Summary:", dataset["train"][i]["summary"], "\n")

{'bill_id': '107_hr2256', 'text': "SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``Border Hospital Survival and Illegal \nImmigrant Care Act''.\n\nSEC. 2. FINDINGS.\n\n    The Congress finds as follows:\n            (1) Immigration is a Federal responsibility.\n            (2) The Immigration and Naturalization Service does not \n        take into custody all aliens who are unlawfully present in the \n        United States.\n            (3) Section 1867 of the Social Security Act (42 U.S.C. \n        1395dd) and State laws require that, if any individual (whether \n        or not lawfully present in the United States) comes to a \n        hospital and the hospital determines that the individual has an \n        emergency medical condition, the hospital must provide either, \n        within the staff and facilities available at the hospital, for \n        such further medical examination and such treatment as may be \n        required to stabilize the medical condition, or

In [28]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [9]:
#Now I will choose a pre-trained model suitable for summarization and will load tokenizer and model
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [14]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

sample_text = dataset["test"][0]["text"]

summary = summarizer(sample_text, max_length=256, min_length=50, do_sample=False)
print(summary[0]["summary_text"])

Device set to use mps:0


{'summary_text': 'This Act may be cited as the \xa0National Science Education Tax  \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0Incentive for Businesses Act of 2007. It applies to science, technology, engineering, and mathematics education at the elementary and secondary school level. It also applies to teacher training expenses and property contributions.'}


In [15]:
#Evaluation

In [18]:
samples = dataset["test"].select(range(5))
results = []
predictions = []
references = []

for i, sample in enumerate(samples):
    original_text = sample["text"]
    original_summary = sample["summary"]

    model_summary = summarizer(original_text, max_length=256, min_length=50, do_sample=False)[0]["summary_text"]

    predictions.append(model_summary)
    references.append(original_summary)
    
    results.append({
    "id": i,
    "original_summary": original_summary,
    "model_summary": model_summary})

for r in results:
    print("="*80)
    print(f"Sample {r['id']}")
    print("\nORIGINAL SUMMARY:\n")
    print(r["original_summary"])
    print("\nMODEL SUMMARY:\n")
    print(r["model_summary"])

Sample 0

ORIGINAL SUMMARY:

National Science Education Tax Incentive for Businesses Act of 2007 - Amends the Internal Revenue Code to allow a general business tax credit for contributions of property or services to elementary and secondary schools and for teacher training to promote instruction in science, technology, engineering, or mathematics .

MODEL SUMMARY:

This Act may be cited as the  National Science Education Tax                 Incentive for Businesses Act of 2007. It applies to science, technology, engineering, and mathematics education at the elementary and secondary school level. It also applies to teacher training expenses and property contributions.
Sample 1

ORIGINAL SUMMARY:

Small Business Expansion and Hiring Act of 2011 - Amends the Internal Revenue Code to allow nongovernmental employers who employ an average of fewer than 100 employees during a taxable year a retained worker tax credit until December 31, 2012, for the lesser of $4,000 or 6.2 of the wages paid t

In [17]:
import evaluate
rouge = evaluate.load("rouge")

In [19]:
rouge_results = rouge.compute(
    predictions = predictions,
    references = references)

rouge_results

{'rouge1': np.float64(0.38548849473378427),
 'rouge2': np.float64(0.17605491751445218),
 'rougeL': np.float64(0.2478265900941004),
 'rougeLsum': np.float64(0.24782659009410035)}